In [ ]:
# Run this cell if output errors occur

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip3 install -U PySuperTuxKart

In [ ]:
%load_ext autoreload
%autoreload 2

# Create pkl training/validation files for imitation learning

# We were given some agents to compete with. Have them play 500 games with each other and record their actions. At the beginning of each game, we randomly set puck's starting velocity and location.

In [ ]:
from itertools import permutations
import subprocess
import random
from tqdm import tqdm

folders_arr = ['agent_1', 'agent_2', 'agent_3', 'agent_4', 'agent_5']
pairs_arr = list(permutations(folders_arr, 2))

for i in tqdm(range(500)):
  index = random.randint(0, len(pairs_arr) - 1)
  item = pairs_arr[index]
  pos_x = random.randint(-30, 30)
  pos_y = random.randint(-30, 30)
  vel_x = random.randint(-20, 20)
  vel_y = random.randint(-20, 20)
  data_f_save_name = "train_files/" + item[0] + '_' + item[1] + "_"\
                       + str(pos_x) + "_" + str(pos_y) + "_" + str(vel_x) + "_" + str(vel_y) + '.pkl'
  print(data_f_save_name)
  subprocess.run(["python3", "-m", "tournament.runner", item[0], item[1], "-s", data_f_save_name,
                  "-f", "600", "-m", "2", "--ball_location", str(pos_x), str(pos_y), "--ball_velocity", str(vel_x), str(vel_y)])

for i in tqdm(range(50)):
  index = random.randint(0, len(pairs_arr) - 1)
  item = pairs_arr[index]
  pos_x = random.randint(-30, 30)
  pos_y = random.randint(-30, 30)
  vel_x = random.randint(-20, 20)
  vel_y = random.randint(-20, 20)
  data_f_save_name = "val_files/" + item[0] + '_' + item[1] + "_"\
                       + str(pos_x) + "_" + str(pos_y) + "_" + str(vel_x) + "_" + str(vel_y) + '.pkl'
  print(data_f_save_name)
  subprocess.run(["python3", "-m", "tournament.runner", item[0], item[1], "-s", data_f_save_name,
                  "-f", "600", "-m", "2", "--ball_location", str(pos_x), str(pos_y), "--ball_velocity", str(vel_x), str(vel_y)])

In [ ]:
from tqdm import tqdm
import numpy as np
import torch.utils.tensorboard as tb
import os
from torch.utils.data import Dataset, DataLoader
import torch
import pickle

# Code for dataset creation

In [ ]:
def limit_period(angle):
    # turn angle into -1 to 1
    return angle - torch.floor(angle / 2 + 0.5) * 2


def extract_features(pstate, soccer_state, opponent_state, team_id):
    """This function is used to perform feature engineering."""

    # features of ego-vehicle
    kart_front = torch.tensor(pstate['kart']['front'], dtype=torch.float32)[[0, 2]]
    kart_center = torch.tensor(pstate['kart']['location'], dtype=torch.float32)[[0, 2]]
    kart_direction = (kart_front-kart_center) / torch.norm(kart_front-kart_center)
    kart_angle = torch.atan2(kart_direction[1], kart_direction[0])

    kart_velocity = np.linalg.norm(pstate['kart']['velocity'])

    # features of soccer
    puck_center = torch.tensor(soccer_state['ball']['location'], dtype=torch.float32)[[0, 2]]
    kart_to_puck_direction = (puck_center - kart_center) / torch.norm(puck_center-kart_center)
    kart_to_puck_angle = torch.atan2(kart_to_puck_direction[1], kart_to_puck_direction[0])

    kart_to_puck_angle_difference = limit_period((kart_angle - kart_to_puck_angle)/np.pi)

    distance_to_puck = np.linalg.norm(kart_center - puck_center)

    # features of opponents
    opponent_center0 = torch.tensor(opponent_state[0]['kart']['location'], dtype=torch.float32)[[0, 2]]
    opponent_center1 = torch.tensor(opponent_state[1]['kart']['location'], dtype=torch.float32)[[0, 2]]

    opponent_velocity0 = np.linalg.norm(opponent_state[0]['kart']['velocity'])
    opponent_velocity1 = np.linalg.norm(opponent_state[1]['kart']['velocity'])

    opponent_distance_to_puck0 = np.linalg.norm(opponent_center0 - puck_center)
    opponent_distance_to_puck1 = np.linalg.norm(opponent_center1 - puck_center)

    kart_to_opponent0 = (opponent_center0 - kart_center) / torch.norm(opponent_center0-kart_center)
    kart_to_opponent1 = (opponent_center1 - kart_center) / torch.norm(opponent_center1-kart_center)

    kart_to_opponent0_angle = torch.atan2(kart_to_opponent0[1], kart_to_opponent0[0])
    kart_to_opponent1_angle = torch.atan2(kart_to_opponent1[1], kart_to_opponent1[0])

    kart_to_opponent0_angle_difference = limit_period((kart_angle - kart_to_opponent0_angle)/np.pi)
    kart_to_opponent1_angle_difference = limit_period((kart_angle - kart_to_opponent1_angle)/np.pi)

    # features of score-line
    goal_line_center = torch.tensor(soccer_state['goal_line'][team_id], dtype=torch.float32)[:, [0, 2]].mean(dim=0)
    opponent_goal_line_center = torch.tensor(soccer_state['goal_line'][np.abs(team_id - 1)], dtype=torch.float32)[:, [0, 2]].mean(dim=0)

    puck_to_goal_line = (goal_line_center-puck_center) / torch.norm(goal_line_center-puck_center)
    puck_to_goal_line_angle = torch.atan2(puck_to_goal_line[1], puck_to_goal_line[0])
    kart_to_goal_line_angle_difference = limit_period((kart_angle - puck_to_goal_line_angle)/np.pi)

    puck_distance_to_the_goal_line = np.linalg.norm(puck_center - goal_line_center)
    puck_opponent_distance_to_goal = np.linalg.norm(puck_center - opponent_goal_line_center)

    features = torch.tensor([kart_center[0], kart_center[1], kart_angle, kart_to_puck_angle, opponent_center0[0],
        opponent_center0[1], opponent_center1[0], opponent_center1[1], kart_to_opponent0_angle, kart_to_opponent1_angle,
        goal_line_center[0], goal_line_center[1],
        puck_to_goal_line_angle, kart_to_puck_angle_difference,
        kart_to_opponent0_angle_difference, kart_to_opponent1_angle_difference,
        kart_to_goal_line_angle_difference,
        kart_velocity, distance_to_puck, puck_center[0], puck_center[1], opponent_velocity0,
        opponent_velocity1, puck_distance_to_the_goal_line, puck_opponent_distance_to_goal
                             ], dtype=torch.float32)

    return features


class Final_Project_Dataset(Dataset):
  def __init__(self, dataset_path):
    data_files = os.listdir(dataset_path)
    data = []
    for file in data_files:
      with open(dataset_path + "/" + file, "rb") as f:
          while True:
              try:
                  data.append(pickle.load(f))
              except EOFError:
                  break

    player_state_data = []
    opponent_state_data = []
    soccer_state_data = []
    self.actions_data = []
    team_id = []

    for entry in (data):
      # Add data for two players
      player_state_data.append(entry['team1_state'][0])
      player_state_data.append(entry['team1_state'][1])

      # Add data for opponents as players
      player_state_data.append(entry['team2_state'][0])
      player_state_data.append(entry['team2_state'][1])

      # Add data for opponents. Duplicate as we have two separate players on our side
      opponent_state_data.append(entry['team2_state'])
      opponent_state_data.append(entry['team2_state'])

      opponent_state_data.append(entry['team1_state'])
      opponent_state_data.append(entry['team1_state'])

      # Add data for soccer
      soccer_state_data.append(entry['soccer_state'])
      soccer_state_data.append(entry['soccer_state'])
      soccer_state_data.append(entry['soccer_state'])
      soccer_state_data.append(entry['soccer_state'])

      # Add actions. Duplicate as we have two separate players on our side
      temp = entry['actions']
      # Actions for the 1 team (players 1 and 3)
      self.actions_data.append([temp[0]['acceleration'], temp[0]['steer'], temp[0]['brake']])
      self.actions_data.append([temp[2]['acceleration'], temp[2]['steer'], temp[2]['brake']])

      # Actions for the 2 team (players 2 and 4)
      self.actions_data.append([temp[1]['acceleration'], temp[1]['steer'], temp[1]['brake']])
      self.actions_data.append([temp[3]['acceleration'], temp[3]['steer'], temp[3]['brake']])

      # Append team_ids
      team_id.append(0)
      team_id.append(0)
      team_id.append(1)
      team_id.append(1)

    self.features = []
    for i in tqdm(range(len(player_state_data))):
      self.features.append(extract_features(player_state_data[i], soccer_state_data[i],
                                      opponent_state_data[i], team_id[i]))
    del data
    del player_state_data
    del soccer_state_data
    del opponent_state_data
    del team_id

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx], self.actions_data[idx]

def load_data(dataset_path, num_workers=0, batch_size=128, shuffle=True):
    dataset = Final_Project_Dataset(dataset_path)
    return DataLoader(dataset, num_workers=num_workers, batch_size=batch_size, shuffle=shuffle, drop_last=True)


# Define model

In [ ]:
class State_Based_Model(torch.nn.Module):
  def __init__(self, input_size=25, sizes=[64, 32, 32, 10]):
    super().__init__()
    layers = []
    for i in range(len(sizes) - 1):
      curr_size = sizes[i]
      layers.append(torch.nn.Linear(input_size, curr_size))
      layers.append(torch.nn.PReLU())
      layers.append(torch.nn.Dropout(0.1))
      input_size = curr_size

    layers.append(torch.nn.Linear(input_size, sizes[-1]))
    self.net = torch.nn.Sequential(*layers)
    self.acceleration_net = torch.nn.Linear(sizes[-1], 1, bias=False)
    self.steer_net = torch.nn.Linear(sizes[-1], 3, bias=False)
    self.brake_net = torch.nn.Linear(sizes[-1], 1, bias=False)

  def forward(self, x):
    net_output = self.net(x)
    acceleration, steer, brake = self.acceleration_net(net_output),\
          self.steer_net(net_output), self.brake_net(net_output)

    acceleration = torch.clamp(acceleration, 0, 1)
    brake = torch.clamp(brake, 0, 1)

    # Set acceleration = 0 if we are braking
    acceleration = acceleration * torch.abs((brake > 0.5).float() - 1.)

    return acceleration, steer, brake

def save_model(model):
    from torch import save
    from os import path
    save(model.state_dict(), "models/model.th")


def load_model(model_path="models/model.th"):
    from torch import load
    from os import path
    r = State_Based_Model()
    r.load_state_dict(torch.load(model_path, map_location='cpu'))
    return r

In [ ]:
def get_mean_std(loader):
  sum = 0
  sum_of_squares = 0
  total_samples = 0

  # Iterate over the DataLoader
  for data, _ in loader:
      sum += torch.sum(data, dim=0)
      sum_of_squares += torch.sum(data ** 2, dim=0)
      total_samples += data.size(0)

  # Calculate the mean and std dev
  mean = sum / total_samples
  std = (sum_of_squares / total_samples - mean ** 2) ** 0.5

  return mean, std

# Train our model with imitation learning

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = load_data("train_files", batch_size=1024, shuffle=True)
val_loader = load_data("val_files", batch_size=2048, shuffle=False)

mse_loss_fn = torch.nn.MSELoss()
bce_loss_fn = torch.nn.BCEWithLogitsLoss()
ce_loss_fn = torch.nn.CrossEntropyLoss()

100%|██████████| 208240/208240 [02:30<00:00, 1380.29it/s]


In [ ]:
train_logger = tb.SummaryWriter("logs/train")
val_logger = tb.SummaryWriter("logs/val")

best_val_loss = 2
best_train_loss = 2

global_step = 0

model = State_Based_Model(25, [64, 32, 32, 10])

optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)

model.to(device)

for epoch in range(1, 30):
  print(f"Epoch: {epoch}")

  model.train()

  loss_arr = []
  acceleration_loss_arr = []
  steer_loss_arr = []
  brake_loss_arr = []

  steer_acc_arr = []
  brake_acc_arr = []

  for x_batch, y_batch in tqdm(train_loader):
    x_batch = x_batch.to(device)

    acceleration_true = y_batch[0].to(device).to(torch.float)
    # Our steering actions are encoded as -1, 0, 1. Convert them to 0, 1, 2
    steer_true = y_batch[1].to(device).to(torch.long).squeeze(dim=1) + 1
    brake_true = y_batch[2].to(device).to(torch.float)

    # Get predictions
    acceleration_pred, steer_pred, brake_pred = model(x_batch)

    # Compute loss
    acceleration_loss = mse_loss_fn(acceleration_pred, acceleration_true)
    steer_loss = ce_loss_fn(steer_pred, steer_true)
    brake_loss = mse_loss_fn(brake_pred, brake_true)
    loss = acceleration_loss + steer_loss + brake_loss

    # log to tensorboard
    train_logger.add_scalar('acceleration_loss', acceleration_loss, global_step=global_step)
    train_logger.add_scalar('steer_loss', steer_loss, global_step=global_step)
    train_logger.add_scalar('brake_loss', brake_loss, global_step=global_step)
    train_logger.add_scalar('loss', loss, global_step=global_step)

    # Backward step
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.01)
    optimizer.step()
    global_step += 1


    acceleration_loss_arr.append(acceleration_loss.detach().cpu())
    steer_loss_arr.append(steer_loss.detach().cpu())
    brake_loss_arr.append(brake_loss.detach().cpu())
    loss_arr.append(loss.detach().cpu())

    steer_acc_arr.append(((torch.nn.functional.softmax(steer_pred, dim=1).argmax(dim=1)) == steer_true).cpu())
    brake_acc_arr.append(((brake_pred > 0.5).float() == brake_true).cpu())


  train_loss = np.mean(loss_arr)
  print(f"Train loss = {train_loss}")

  acceleration_loss = np.mean(acceleration_loss_arr)
  print(f"Acceleration loss = {acceleration_loss}")

  steer_loss = np.mean(steer_loss_arr)
  print(f"Steer loss = {steer_loss}")

  brake_loss = np.mean(brake_loss_arr)
  print(f"Brake loss = {brake_loss}")

  steer_acc = np.mean(steer_acc_arr)
  print(f"Steer acc = {steer_acc}")

  brake_acc = np.mean(brake_acc_arr)
  print(f"Brake acc = {brake_acc}")


  # Perform validation
  model.eval()

  loss_arr = []
  acceleration_loss_arr = []
  steer_loss_arr = []
  brake_loss_arr = []

  steer_acc_arr = []
  brake_acc_arr = []

  for x_batch, y_batch in tqdm(val_loader):
    x_batch = x_batch.to(device)

    acceleration_true = y_batch[0].to(device).to(torch.float)
    # Our steering actions are encoded as -1, 0, 1. Convert them to 0, 1, 2
    steer_true = y_batch[1].to(device).to(torch.long).squeeze(dim=1) + 1
    brake_true = y_batch[2].to(device).to(torch.float)

    acceleration_pred, steer_pred, brake_pred = model(x_batch)

    acceleration_loss = mse_loss_fn(acceleration_pred, acceleration_true)
    steer_loss = ce_loss_fn(steer_pred, steer_true)
    brake_loss = mse_loss_fn(brake_pred, brake_true)
    loss = acceleration_loss + steer_loss + brake_loss

    val_logger.add_scalar('acceleration_loss', acceleration_loss, global_step=global_step)
    val_logger.add_scalar('steer_loss', steer_loss, global_step=global_step)
    val_logger.add_scalar('brake_loss', brake_loss, global_step=global_step)
    val_logger.add_scalar('loss', loss, global_step=global_step)

    acceleration_loss_arr.append(acceleration_loss.detach().cpu())
    steer_loss_arr.append(steer_loss.detach().cpu())
    brake_loss_arr.append(brake_loss.detach().cpu())
    loss_arr.append(loss.detach().cpu())

    steer_acc_arr.append(((torch.nn.functional.softmax(steer_pred, dim=1).argmax(dim=1)) == steer_true).cpu())
    brake_acc_arr.append(((brake_pred > 0.5).float() == brake_true).cpu())

  val_loss = np.mean(loss_arr)
  print(f"Val loss = {val_loss}")

  acceleration_loss = np.mean(acceleration_loss_arr)
  print(f"Acceleration loss = {acceleration_loss}")

  steer_loss = np.mean(steer_loss_arr)
  print(f"Steer loss = {steer_loss}")

  brake_loss = np.mean(brake_loss_arr)
  print(f"Brake loss = {brake_loss}")

  steer_acc = np.mean(steer_acc_arr)
  print(f"Steer acc = {steer_acc}")

  brake_acc = np.mean(brake_acc_arr)
  print(f"Brake acc = {brake_acc}")


  # Save Model if the chosen metric is better
  if val_loss < best_val_loss:
    save_model(model)
    print("Model saved")
    best_val_loss = val_loss


  # LR scheduler step
  if epoch % 3 == 0:
    print("LR_Scheduler step")
    lr_scheduler.step()
    print(lr_scheduler.get_last_lr())


Epoch: 1


100%|██████████| 1958/1958 [00:40<00:00, 48.70it/s]


Train loss = 0.7116305232048035
Acceleration loss = 0.056832075119018555
Steer loss = 0.5694422721862793
Brake loss = 0.08535612374544144
Steer acc = 0.7576389332226762
Brake acc = 0.8789376715717568


100%|██████████| 101/101 [00:01<00:00, 86.58it/s]


Val loss = 0.5171760320663452
Acceleration loss = 0.018362050876021385
Steer loss = 0.4603717625141144
Brake loss = 0.03844226896762848
Steer acc = 0.8110835009282178
Brake acc = 0.9470964186262376
Model saved
Epoch: 2


...

Val loss = 0.36163246631622314
Acceleration loss = 0.015489479526877403
Steer loss = 0.3108453154563904
Brake loss = 0.035297635942697525
Steer acc = 0.8827738242574258
Brake acc = 0.9507754486386139
Epoch: 29


100%|██████████| 1958/1958 [00:46<00:00, 42.27it/s]


Train loss = 0.38312065601348877
Acceleration loss = 0.01894860900938511
Steer loss = 0.32310035824775696
Brake loss = 0.04107171297073364
Steer acc = 0.8815371831907559
Brake acc = 0.9461773413559755


100%|██████████| 101/101 [00:01<00:00, 74.61it/s]

Val loss = 0.3603377342224121
Acceleration loss = 0.015444085001945496
Steer loss = 0.30967944860458374
Brake loss = 0.03521424159407616
Steer acc = 0.8842289990717822
Brake acc = 0.9510268409653465
Model saved


In [ ]:
# !rm -r training_files/.ipynb_checkpoints/

rm: cannot remove 'training_files/.ipynb_checkpoints/': No such file or directory


In [ ]:
# !rm -r logs

In [ ]:
%load_ext tensorboard

In [ ]:
import torch.utils.tensorboard as tb

%tensorboard --logdir {'logs'}

# Use REINFORCE to imporve our model

In [ ]:
import copy
import subprocess
import random
from tqdm import tqdm
import pickle
from torch.distributions import Bernoulli, Categorical

In [ ]:
!cp scripts/model.pt state_agent/model.pt

In [ ]:
model = load_model('/models/model.th')

In [ ]:
average_return_logger = tb.SummaryWriter("logs/average_return")

In [ ]:
folders_arr = ['agent_1', 'agent_2', 'agent_3', 'agent_4', 'agent_5']

batch_size = 512

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

global_step = 0

for epoch in tqdm(range(20)):
  print(f"Epoch = {epoch}")

  # Clear the directory used from any files from previous runs
  subprocess.run(["rm", "-r", "reinforce_files/"])
  subprocess.run(['mkdir', 'reinforce_files'])

  # Have our agent play 40 random games
  for i in range(40):
    index = random.randint(0, len(folders_arr) - 1)
    player_1 = "state_agent"
    player_2 = folders_arr[index]

    # This variable defines the side of our player in the game (red or blue)
    player_1_pos = random.randint(0, 1)

    pos_x = random.randint(-30, 30)
    pos_y = random.randint(-30, 30)
    vel_x = random.randint(-20, 20)
    vel_y = random.randint(-20, 20)

    if player_1_pos == 0:
      data_f_save_name = "reinforce_files/" + player_1 + '_' + player_2 + "_"\
                    + str(pos_x) + "_" + str(pos_y) + "_" + str(vel_x) + "_" \
                    + str(vel_y) + '.pkl'
      subprocess.run(["python3", "-m", "tournament.runner", player_1, player_2, "-s", data_f_save_name,
                      "-f", "600", "-m", "2", "--ball_location", str(pos_x), str(pos_y), "--ball_velocity", str(vel_x), str(vel_y)])
    else:
      data_f_save_name = "reinforce_files/" + player_2 + '_' + player_1 + "_"\
              + str(pos_x) + "_" + str(pos_y) + "_" + str(vel_x) + "_" \
              + str(vel_y) + '.pkl'
      subprocess.run(["python3", "-m", "tournament.runner", player_2, player_1, "-s", data_f_save_name,
                  "-f", "600", "-m", "2", "--ball_location", str(pos_x), str(pos_y), "--ball_velocity", str(vel_x), str(vel_y)])


  data_files = os.listdir("final/reinforce_files/")
  data = []
  for file in data_files:
    with open("reinforce_files" + "/" + file, "rb") as f:
        while True:
            try:
                data.append(pickle.load(f))
            except EOFError:
                break

  player_state_data = []
  opponent_state_data = []
  soccer_state_data = []
  actions_data = []
  player_pos_arr = []

  for entry in data:
    # Add player/opponent data depending on the position of our agents in the game

    if 'drift' in entry['actions'][0].keys():
      player_pos = 0
    else:
      player_pos = 1

    player_pos_arr.append(player_pos)

    if player_pos == 0:
      player_state_data.append(entry['team1_state'])
      opponent_state_data.append(entry['team2_state'])
    else:
      player_state_data.append(entry['team2_state'])
      opponent_state_data.append(entry['team1_state'])

    soccer_state_data.append(entry['soccer_state'])
    actions_data.append(entry['actions'])

  features = []
  returns = []
  actions = []

  # Get initial features for our first and second players
  initial_features_1 = extract_features(player_state_data[0][0],
                                             soccer_state_data[0], opponent_state_data[0], player_pos_arr[0])
  initial_features_2 = extract_features(player_state_data[0][1],
                                             soccer_state_data[0], opponent_state_data[0], player_pos_arr[0])

  prev_puck_goal_distance_1 = initial_features_1[-2]
  prev_puck_player_distance_1 = initial_features_1[-7]
  prev_puck_goal_distance_2 = initial_features_2[-2]
  prev_puck_player_distance_2 = initial_features_2[-7]

  player_state_data = player_state_data[1:]
  opponent_state_data = opponent_state_data[1:]
  soccer_state_data = soccer_state_data[1:]
  actions_data = actions_data[1:]
  player_pos_arr = player_pos_arr[1:]

  for i in range(len(player_state_data)):
    player_pos = player_pos_arr[i]
    curr_features_1 = extract_features(player_state_data[i][0],
                                             soccer_state_data[i], opponent_state_data[i], player_pos)
    curr_features_2 = extract_features(player_state_data[i][1],
                                             soccer_state_data[i], opponent_state_data[i], player_pos)
    curr_puck_goal_distance_1 = curr_features_1[-2]
    curr_puck_player_distance_1 = curr_features_1[-7]

    curr_puck_goal_distance_2 = curr_features_1[-2]
    curr_puck_player_distance_2 = curr_features_1[-7]

    # Experiment with the rewards given to our agents
    reward_1 = (prev_puck_goal_distance_1 - curr_puck_goal_distance_1) # + (prev_puck_player_distance_1 - curr_puck_player_distance_1)
    reward_2 = (prev_puck_goal_distance_2 - curr_puck_goal_distance_2) # + (prev_puck_player_distance_2 - curr_puck_player_distance_2)

    if reward_1 != 0:
      returns.append(reward_1)
      features.append(curr_features_1)
      actions.append(((actions_data[i][player_pos]['acceleration'] > 0.5).float(),
                (actions_data[i][player_pos]['steer'] + 1.),
                (actions_data[i][player_pos]['brake'] > 0.5).float()))

    if reward_2 != 0:
      returns.append(reward_2)
      features.append(curr_features_2)
      actions.append(((actions_data[i][player_pos+2]['acceleration'] > 0.5).float(),
                (actions_data[i][player_pos+2]['steer'] + 1.),
                (actions_data[i][player_pos+2]['brake'] > 0.5).float()))


    prev_puck_goal_distance_1 = curr_puck_goal_distance_1
    prev_puck_player_distance_1 = curr_puck_player_distance_1
    prev_puck_goal_distance_2 = curr_puck_goal_distance_2
    prev_puck_player_distance_2 = curr_puck_player_distance_2



  number_of_iterations_to_train = int(len(features) / batch_size)
  print(f"We are going to train for {number_of_iterations_to_train} iterations")

  returns = torch.as_tensor(returns, dtype=torch.float32).cuda()
  actions = torch.as_tensor(actions).cuda()
  features = torch.stack(features).cuda()

  returns = (returns - returns.mean()) / returns.std()
  # returns = returns / returns.std()

  model.train().cuda()
  avg_expected_log_return = []
  for i in range(number_of_iterations_to_train):
    batch_ids = torch.randint(0, len(returns), (batch_size, )).cuda()

    batch_returns = returns[batch_ids]
    batch_actions = actions[batch_ids]
    batch_features = features[batch_ids]

    acceleration, steer, brake = model(batch_features)

    pi_acceleration = Bernoulli(logits=acceleration[:, 0])
    pi_steer = Categorical(logits=steer)
    pi_brake = Bernoulli(logits=brake[:, 0])

    expected_log_return = ((pi_acceleration.log_prob(batch_actions[:, 0]) * batch_returns).mean() +
                          (pi_steer.log_prob(batch_actions[:, 1]) * batch_returns).mean() +
                          (pi_brake.log_prob(batch_actions[:, 2]) * batch_returns).mean()) / 3

    optimizer.zero_grad()
    (-expected_log_return).backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.01)
    optimizer.step()
    avg_expected_log_return.append(float(expected_log_return))

    average_return_logger.add_scalar('average_return', float(expected_log_return), global_step=global_step)
    global_step += 1

  print(f"Average expected log return {np.mean(avg_expected_log_return)}")

  model.eval()
  model.cpu()
  batch_features = batch_features.to('cpu')

  traced_script_module = torch.jit.trace(model, batch_features)
  traced_script_module.save("model.pt")

# Get torch jit script

In [ ]:
model = load_model()
model.eval()
x_batch = x_batch.to('cpu')

traced_script_module = torch.jit.trace(model, x_batch)
traced_script_module.save("model.pt")

In [ ]:
!cp scripts/model.pt state_agent/

# Test new agent

In [ ]:
import moviepy.editor
moviepy.editor.ipython_display("my_video.mp4")